# 학업탄력성 영향요인 연구
@author: sjh

- 전체 데이터 로드 및 가공

## 1. Load

In [1]:
import os
import copy
import json
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# unicode minus를 사용하지 않기 위한 설정 (minus 깨짐현상 방지)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["figure.autolayout"] = True

# 설치된 폰트 출력
import matplotlib.font_manager as fm
font_list = [font.name for font in fm.fontManager.ttflist]
plt.rcParams['font.family'] = 'Malgun Gothic'

import seaborn as sns
sns.set_style("darkgrid")


App_dir = os.getcwd()
Data_dir = os.path.join(App_dir, 'data')
Result_dir = os.path.join(App_dir, 'rs')
print('>> Current WD: ', App_dir)

>> Current OS:  win32
>> Current WD:  c:\Users\jhun1\Dev\Research\MixedRF


## 2. Preprocessing
- 앞선 처리 데이터 불러오기
- Load.py로 처리함

In [4]:
# loadedData = {'SK': [
#             pd.read_excel(os.path.join(BASE_DIR,'data', 'cleanedData(SK).xlsx'), sheet_name='stu'),
#             pd.read_excel(os.path.join(BASE_DIR,'data', 'cleanedData(SK).xlsx'), sheet_name='sch'),
#             pd.read_excel(os.path.join(BASE_DIR,'data', 'cleanedData(SK).xlsx'), sheet_name='tch'),
            
#             ],
#         'US': [
#             pd.read_excel(os.path.join(BASE_DIR,'data', 'cleanedData(US).xlsx'), sheet_name='stu'),
#             pd.read_excel(os.path.join(BASE_DIR,'data', 'cleanedData(US).xlsx'), sheet_name='sch'),
#             pd.read_excel(os.path.join(BASE_DIR,'data', 'cleanedData(US).xlsx'), sheet_name='tch'),
#             ]}
with open(os.path.join(App_dir, 'data', 'cleaned.pkl'), 'rb') as f:
    loadedData = pickle.load(f)

In [ ]:
# class Preprocessing:
    # def __init__(self, LoadedData, codeBook, dummyCodeBook, PV_var):

class Preprocessing:
    def __init__(self, LoadedData, codeBook, PV_var):
        self.data = LoadedData
        BASE_DIR = r'C:\Users\jhun1\Dropbox\[2]Project\[혼합효과 랜덤포레스트_2022]' #!#
        self.cb = pd.read_excel(os.path.join(BASE_DIR, 'drive-download-20220816T053902Z-001', codeBook), sheet_name='변수선택(1213)')
        self.PV_var = PV_var

        #!# testBook 기능 확인 필요함, 이전 파일 참고
        self.testBook = {
                    'read/math/sci_1': 'PV1MATH PV1READ PV1SCIE'.split(),
                    'read/math_1': 'PV1MATH PV1READ'.split(),
                    'read/math_10': 'PV1MATH PV2MATH PV3MATH PV4MATH PV5MATH PV6MATH PV7MATH PV8MATH PV9MATH PV10MATH PV1READ PV2READ PV3READ PV4READ PV5READ PV6READ PV7READ PV8READ PV9READ PV10READ'.split(),
                    'read_1': ['PV1READ'],
                    'read_10': 'PV1READ PV2READ PV3READ PV4READ PV5READ PV6READ PV7READ PV8READ PV9READ PV10READ'.split()
                }
        self.nation_real_name = {'SK': '대한민국', 'US': '미국'} 

        self.valid_data = {}
        self._2_joined = {}
        self._3_dropNa = {}
        self._4_ESCS = {'full': {}, 'sliced': {}} # 여기서 데이터 갈라야함
        self._5_shouldBeCal = {}
        self.finalRS = {}
        
        self.rs_deescriptive_Full = pd.DataFrame()
        self.rs_deescriptive_SK = pd.DataFrame()
        self.rs_deescriptive_US = pd.DataFrame()
        

    ### Needs1. is selected variable contained in both dataset
    #!# load.py의 함수와 병합하기
    def noDataColumn(self):
        r"""
        - drop feature, if it has too many NA (over 80%) 
        """
        toDrop = {}
        
        for nationName, nationalData in self.data.items():
            toDrop[nationName] = []
            for idx, (label, inputDf) in enumerate(zip('stu sch tch'.split(), nationalData)):
                if label == 'tch': #!# 굳이 tch을 살펴보지 않은 이유는?
                    pass
                else:
                    for column in inputDf.columns:
                        if inputDf[column].isna().sum() > (inputDf.shape[0] * 0.8):
                            print('>>> too much NA: ', column)
                            toDrop[nationName].append(column)
                            continue
                        
                        elif 'missing' in inputDf[column].values:
                            print('>>> missing: ', column)
                            toDrop[nationName].append(column)
                            continue
                        
                        else:
                            continue

            for nation, grouped_data in self.data.items():
                self.valid_data[nation] = []
                for idx, each_group_data in enumerate(grouped_data):
                    if idx == 0 : # in case of 'student'
                        #!# 코드북고 함께 확인필요, 굳이 RESPECT열을 떨군 이유가?
                        self.valid_data[nation].append(each_group_data.drop('PERSPECT', axis=1))
                    else: 
                        self.valid_data[nation].append(each_group_data)
                        
        return toDrop


    def Join(self):
        r"""
        join student and school dataframe
        #!# 교사데이터는 어디서 합치는가
        """
        print('\n\n>>>> 2. Join DataFrame')

        for nationalName, inputNational in self.valid_data.items():
            # print('>> join nation: ', nationalName)
            df_student = inputNational[0].copy()
            df_school = inputNational[1].copy()
            
            df_student.reset_index(drop=True, inplace=True)
            
            rs = copy.deepcopy(df_student)
            before = df_student.shape


            df_school.drop(['CNTRYID', 'CNT'], axis=1, inplace=True)
            if df_school.index.name != 'CNTSCHID':
                df_school.set_index('CNTSCHID', drop=True, inplace=True)
            
            if df_school.shape[1] == 0:
                print('>> school data is empty')
                pass
            else:
                for idx in tqdm(df_student.index, desc=">> mapping"):
                    toBeInput = df_school.loc[idx, 'CNTSCHID'].values # 학생 데이터에 들어가야할 학교 데이터 찾기
                    assert len(toBeInput) == df_school.shape[1]
                    
                    toBeInput_T = toBeInput.reshape(1, 8)
                    rs.loc[idx, list(df_school.columns)] = toBeInput_T[0]
            
                after = rs.shape
                print('>>>> Bef: ', before, '....', 'Aft: ', after)
                assert 'EDUSHORT' in rs.columns #!# 왜 필요할까

            self._2_joined[nationalName] = rs
        return rs

    def DropStudent(self, na_threshold: int, is_visualize=True):
        r"""
        drop student who have too many NA value
        Parameters
        ----------
        na_threshold: int
            drop student who have NA value above this threshold
        """
        print('\n>>>> 3. Verify na and Drop student')
        def column_wise_NA(inputData) -> dict:
            r"""generate column-wise NA ratio"""
            if type(inputData) == dict:
                merged = pd.concat([inputData['SK'], inputData['US']])
                assert merged.shape[0] == inputData['SK'].shape[0] + inputData['US'].shape[0]
            
            elif type(inputData) == pd.DataFrame:
                merged = copy.deepcopy(inputData)
            
            else:
                raise TypeError('dictionary or pd.DataFrame is allowed')
                

            describeDF = merged.describe().T
            describeDF['NA_ratio'] = round(100 - describeDF['count']/merged.shape[0]*100, 2)

            newColumnOrder = [describeDF.columns[0], 'NA_ratio'] + list(describeDF.columns[1:-1])
            describeDF= describeDF[newColumnOrder]
            return describeDF

        # 각 학생별로 데이터 검수
        def row_wise_NA(inputData: dict, is_visualize: bool, na_threshold: int) -> dict:
            r"""calculate NA ratio per student"""
            merged = pd.concat([inputData['SK'], inputData['US']])
            assert merged.shape[0] == inputData['SK'].shape[0] + inputData['US'].shape[0]

            for_histogram = {}
            rs = {}
            for label, data in zip(['full', 'SK', 'US'], [merged, inputData['SK'], inputData['US']]):
                for_histogram[label] = []
                to_drop = []

                for i in range(len(data.index)) :
                    na_cnt = data.iloc[i].isnull().sum()
                    na_ratio = round((na_cnt/data.shape[1]) * 100, 0)
                    for_histogram[label].append(na_ratio)
                    if na_cnt > na_threshold:
                        to_drop.append(i)
                print(f'>> NA drop of {label}: ', len(to_drop))
                rs[label] = data.drop(to_drop, axis=1)

            if is_visualize == True:
                fig = plt.figure(figsize=(17,6))

                plt.subplot(1, 3, 1)
                plt.hist(for_histogram['full'])
                plt.title('\n전체 데이터\n')
                plt.xlabel('\n전체 변수 대비 결측비율(%)\n')
                plt.ylabel('빈도')
                
                plt.subplot(1, 3, 2)
                plt.hist(for_histogram['SK'])
                plt.title('\nSouth Korea\n')
                plt.xlabel('\n전체 변수 대비 결측비율(%)\n')
                plt.ylabel('빈도')
                
                plt.subplot(1, 3, 3)
                plt.hist(for_histogram['US'])
                plt.title('\nUnited States\n')
                plt.xlabel('\n전체 변수 대비 결측비율(%)\n')
                plt.ylabel('빈도')

                plt.savefig(os.path.join(Data_dir, f'NA_ratio.png'))
                plt.show()


            return rs
        
        #!# 이 부분은 sequential한 단계에서 빠져야할 것 같음, EDA에 가까움
        # visualize와 같이, debug 단계만 사용되면 되고, 나머지에서는 굳이 계산할 필요가 없음
        self.rs_deescriptive_Full = column_wise_NA(self._2_joined)
        self.rs_deescriptive_SK = column_wise_NA(self._2_joined['SK'])
        self.rs_deescriptive_US = column_wise_NA(self._2_joined['US'])

        clean_data_using_rowwise_NA = row_wise_NA(self._2_joined, na_threshold=na_threshold, is_visualize=is_visualize)
        self._3_dropNa['SK'] = clean_data_using_rowwise_NA['SK']
        self._3_dropNa['US'] = clean_data_using_rowwise_NA['US']
        return self._3_dropNa
        
        
    def slice_by_ESCS(self, acad_threshold: int, is_visualize=True) -> dict:
        r"""
        calculate ESCS variable and devide dataset by full and sliced
        
        Parameters
        ----------
        acad_threshold: int
            academic score thrshold
        """
        print('\n>>>> 4. Slicing data by ESCS')
        
        def thresholdCalculator(data: dict,
                                PV_var: int,
                                acad_threshold: int):
            r"""calculate 2 kinds of threshold,
            1. academic score
            2. ESCS
            """
            assert type(PV_var) == int, 'insert valid PV_var type'
            assert type(acad_threshold) == int, 'insert valid threshold type'
            assert (PV_var > 0) and (PV_var < 11), print('>> Error__PV_var: ', PV_var)

            threshold_dict = {'SK': {'academic_score': acad_threshold}, 'US': {'academic_score': acad_threshold}}
            targetColumn = ['PV'+ str(PV_var) + 'READ']
            rs = data.copy()

            for nationalName, inputNational in data.items():
                rs[nationalName]['AcademicScore'] = inputNational.loc[:, targetColumn].mean(axis=1)
                threshold_dict[nationalName]['escs_score'] = inputNational['ESCS'].quantile(0.25)

            return threshold_dict, rs
        
        def slice_by_escs(data: dict, escsThreshold: dict) -> dict:
            r"""slice data by escs score
            """
            assert type(data) == dict, 'insert valid data'
            assert type(escsThreshold) == dict, 'insert valid threshold'

            rs = {'SK': pd.DataFrame(), 'US': pd.DataFrame()}
            for nationalName, inputNational in data.items():
                before = inputNational.shape[0]
                toDrop = []
                for idx, val in zip(inputNational['ESCS'].index, inputNational['ESCS'].values):
                    if val < escsThreshold[nationalName]['escs_score']:
                        continue
                    else:
                        toDrop.append(idx)
                
                rs[nationalName] = inputNational.drop(toDrop, axis=0)
                after = rs[nationalName].shape[0]
                print('>> before: ', before, '>> after: ', after)
            
            return rs


        def labeling_resilient( 
                            data, # 전체 Full, escs 하위 25%로 데이터셋이 2개로 나뉘므로 인풋을 줘야함
                            option: str, # full: 전체 데이터, sliced: 잘린 데이터
                            threshold_info: dict
                            ):
            r"""
            Parameters
            ----------
            option: str
                full or sliced

            if condition1: using academic score
                & condition2: using escs score
                full: condition1 & condition2
                sliced: condition1
                - since sliced data already sliced by escs score
            """
            if (option == 'full') or (option == 'sliced'): pass
            else:
                raise ValueError('input valid option args')
                
            assert type(threshold_info) == dict
            rs = {'SK': pd.DataFrame(), 'US': pd.DataFrame()}
            
            
            for nationalName, inputNational in data.items():
                threshold_acad = threshold_info[nationalName]['academic_score']
                threshold_escs = threshold_info[nationalName]['escs_score']
                
                iamResilient = []
                for idx in inputNational.index:
                    val_acad = inputNational.loc[idx, 'AcademicScore']
                    val_escs = inputNational.loc[idx, 'ESCS']
                    if option == 'full':
                        if (val_acad > threshold_acad) and (val_escs < threshold_escs):
                            iamResilient.append(1)
                        else: iamResilient.append(0)
                    elif option == 'sliced': #!# 여기서 에러 날수도..?
                        if val_acad > threshold_acad:
                            iamResilient.append(1)
                        else: iamResilient.append(0)

                inputNational['resilient'] = iamResilient
                rs[nationalName] = inputNational.copy()

            return rs
        
        def table_resilient_ratio(data: dict) -> list:
            r"""회복탄력성을 지닌 학생의 전체 대비 비율을 계산함"""
            count_ratio = {'SK': [], 'US': []} #!# count, ratio로 dict형태인게 더 나아보임
            for nationalName in count_ratio.keys():
                total = data[nationalName].shape[0]
                is_resilient = data[nationalName]['resilient'].values

                resilientCount = [x for x in is_resilient if x == 1]
                resilientRatio = round(len(resilientCount)/total * 100, 2)
                count_ratio[nationalName].append(len(resilientCount))
                count_ratio[nationalName].append(resilientRatio)
                print(f'>> 회복탄력성 보유 학생수({nationalName}): ', len(resilientCount), f'({resilientRatio})%')
            return count_ratio

        def visualize(data: dict, option: str, threshold_info: dict,
                    figName: str):
            r"""visualize threshold and ratio of sample distribution
            option: str
                full or sliced
            figName: str
                title of figure
            """
            fig = plt.figure(figsize=(17,9))
            for IDX, (nationalName, inputNational) in enumerate(data.items()):

                plt.subplot(2, 2, 2*IDX+1)
                plt.hist(inputNational['AcademicScore'])
                plt.title(f'\n학업성취{self.nation_real_name[nationalName]}\n')
                plt.xlabel('\n점수\n')
                plt.axvline(threshold_info[nationalName]['academic_score'], color='r', linewidth=1, linestyle='--')
                
                plt.subplot(2, 2, 2*IDX+2)
                plt.hist(inputNational['ESCS'])
                plt.title(f'\n사회문화경제{self.nation_real_name[nationalName]}\n')
                plt.xlabel('\n점수\n')
                if option=='full':
                    plt.axvline(threshold_info[nationalName]['escs_score'], color='r', linewidth=1, linestyle='--')

                
            plt.savefig(os.path.join(Result_dir, f'{figName}_{option}.png'))
            plt.show()
        
        ## 1. calculate threshold value: 따로 메소드로 빼도 될듯
        threshold_info, newData_dict = thresholdCalculator(self._3_dropNa,
                                                            PV_var = self.PV_var,
                                                            acad_threshold = acad_threshold) ##!#!## 학업성취 코딩 방법을 바꿀 때 여기 arg를 조정
        
        
        ## 2. slice
        self._4_ESCS['full'] = copy.deepcopy(newData_dict) # no drop case, so just copied
        self._4_ESCS['sliced'] = slice_by_escs(newData_dict, escsThreshold = threshold_info)
        assert type(self._4_ESCS['full']) == dict, print(self._4_ESCS['full'])


        ## 3. labeling resilient student
        self._4_ESCS['full'] = labeling_resilient(inputData=self._4_ESCS['full'], 
                                                option = 'full',
                                                AcademicThreshold= threshold_info)
        self._4_ESCS['sliced'] = labeling_resilient(inputData=self._4_ESCS['sliced'], 
                                                option = 'sliced',
                                                AcademicThreshold= threshold_info)
        
        #!# 앞선 visualize 단계와 마찬가지임, debug단계가 아니면 확인하지 않는 부분임
        resilientCount_Ratio_full = table_resilient_ratio(data=self._4_ESCS['full'])
        resilientCount_Ratio = table_resilient_ratio(data=self._4_ESCS['sliced'])

        ## 4. visualize resilient student
        if is_visualize == True:
            visualize(self._4_ESCS['full'], option='full', figName='읽10', threshold_info= threshold_info)
            visualize(self._4_ESCS['sliced'], option = 'sliced', figName ='읽10(target paper)', threshold_info= threshold_info)
        
        return resilientCount_Ratio

    
    # should be calculated 변수들 계산하는 것임
    def shouldBeCalculated(self):
        # print('\n\n>>>> 6. Should Be Calculated')
        
        def schoolMean(inputDf, whichVar):
            assert type(whichVar) == list
            outputMean = {}
            for sch_id in inputDf['CNTSCHID'].values:
                if sch_id in outputMean.keys():
                    continue
                
                else:
                    temp1 = inputDf[inputDf['CNTSCHID'] == sch_id]
                    temp2 = temp1.loc[:, whichVar] 
                    assert len(temp2.columns) == len(whichVar)
                    meanVal = np.nanmean(temp2.values)
                    assert type(meanVal) == np.float64, print('Error : ', type(meanVal))

                    outputMean[sch_id] = meanVal
            
            return outputMean
        
        
        def meanMapping(inputColumn, mean_dict):
            outputLS = []
            for idx, sch_id in enumerate(inputColumn.values):
                outputLS.append(mean_dict[sch_id])

            return outputLS
        

        def matching(inputData, codeBook):
            output = copy.deepcopy(inputData)
            shouldBeCal = self.cb[self.cb['file name'] == 'should be caculated']
            assert len(shouldBeCal) == 2, print('Error: check self.cb')
            for national in output.keys():
                beforeShape = output[national].shape[1]
                
                calVal = list(codeBook[codeBook['categories'] == 'resilient status']['NAME'])
                calVal.remove('ESCS')
                
                for variable in shouldBeCal['NAME'].values:
                
                    if variable == 'AVG_S_TEST':    
                        mean_dict = schoolMean(output[national], calVal)
                        
                    elif variable == 'AVG_S_ESCS':
                        mean_dict = schoolMean(output[national], ['ESCS'])

                    #평균 dict 활용해서 매칭 진행
                    outputLS = meanMapping(output[national]['CNTSCHID'], mean_dict)
                    assert len(outputLS) == output[national].shape[0], print('Error: ', len(outputLS))
                    output[national][variable] = outputLS # 학교 데이터이므로, 학교에 맞춰서 추가하기

                afterShape = output[national].shape[1]
                assert afterShape - beforeShape == 2, print('Beofre: ', beforeShape, ' ... ', 'After: ', afterShape)
            
            return output
        
        if 'should be calculated' in self.cb['file name'].values:
            self._5_shouldBeCal['full'] = matching(self._4_ESCS['full'], codeBook = self.cb)
            self._5_shouldBeCal['sliced'] = matching(self._4_ESCS['sliced'], codeBook = self.cb)
        else:
            self._5_shouldBeCal['full'] = copy.deepcopy(self._4_ESCS['full'])
            self._5_shouldBeCal['sliced'] = copy.deepcopy(self._4_ESCS['sliced'])
            

    
    def AdjustMinor(self):
        r"""마이너한 것들을 조정하기 위함"""
        
        def Merge(inputData):
            r"""두 데이터를 나라 row를 만들고, 합치기 위함"""
            output = pd.concat([inputData['SK'], inputData['US']], axis=0)
            assert inputData['SK'].shape[0] + inputData['US'].shape[0] == output.shape[0]

            dropAcademic = ['CNTRYID', 'AcademicScore']
            for column in output.columns:
                if 'PV' in column:
                    dropAcademic.append(column)

            output.drop(dropAcademic, axis=1, inplace=True)
            # print('>> columns: ', output.columns)
            return output

        def columnOrder(inputData,
                        important_columns=['resilient']):
            r"""spss 편하도록, 주요 변수들을 앞으로 빼는 작업"""
            column_ID = ['CNT', 'CNTSCHID', 'CNTSTUID']

            inputData.set_index(column_ID+important_columns, inplace=True)
            inputData.reset_index(inplace=True)

            return inputData

        self.finalRS['full'] = Merge(self._5_shouldBeCal['full'])
        self.finalRS['sliced'] = Merge(self._5_shouldBeCal['sliced'])

        self.finalRS['full'] = columnOrder(self.finalRS['full'])
        self.finalRS['sliced'] = columnOrder(self.finalRS['sliced'])

    def Save(self):
        with pd.ExcelWriter(os.path.join(Result_dir, f'preprocessing{self.PV_var}.xlsx')) as writer:
            self.finalRS['sliced'].to_excel(writer, sheet_name='sliced', index=False)


# processor = Preprocessing(LoadedData=loadedData, codeBook=Loader.cb, PV_var=1)
# processor.noDataColumn()
# processor.Join()
# processor.DropStudent(na_threshold=30, is_visualize=False)
# processor.slice_by_ESCS(480, is_visualize=False) # Lv4: 553, Lv3: 480, Lv2: 407
# processor.shouldBeCalculated()
# processor.AdjustMinor()
# processor.Save()

In [8]:
#!# 이거 자체를 하나의 fnc으로 빼야함

plot_sk = pd.DataFrame(index=range(1, 11), columns=['count', 'ratio'])
plot_us = pd.DataFrame(index=range(1, 11), columns=['count', 'ratio'])

for idx in tqdm(range(1, 11)):
    print('>>>> ', idx)
    processor = Preprocessing(LoadedData=loadedData, codeBook='PISA2018_CODEBOOK (변수선택-공유).xlsx', PV_var=idx)
    processor.noDataColumn()
    processor.Join()
    processor.DropStudent(na_threshold=30, is_visualize=False)
    processor.slice_by_ESCS(480, is_visualize=False) # Lv4: 553, Lv3: 480, Lv2: 407
    
    processor.shouldBeCalculated()
    processor.AdjustMinor()
    processor.Save()

    plot_sk.loc[idx, 'count'] = resilientCount_Ratio['SK'][0]
    plot_sk.loc[idx, 'ratio'] = resilientCount_Ratio['SK'][1]
    plot_us.loc[idx, 'count'] = resilientCount_Ratio['US'][0]
    plot_us.loc[idx, 'ratio'] = resilientCount_Ratio['US'][1]


display(plot_sk)

  0%|          | 0/10 [00:00<?, ?it/s]

>>>>  1

>>>> 4. Slicing data by ESCS
>  {'SK': [876, 52.96], 'US': [483, 41.14]}


 10%|█         | 1/10 [00:04<00:41,  4.59s/it]

>>>>  2

>>>> 4. Slicing data by ESCS
>  {'SK': [857, 51.81], 'US': [475, 40.46]}


 20%|██        | 2/10 [00:09<00:36,  4.57s/it]

...

>>>>  9

>>>> 4. Slicing data by ESCS
>  {'SK': [859, 51.93], 'US': [485, 41.31]}


 90%|█████████ | 9/10 [00:40<00:04,  4.40s/it]

>>>>  10

>>>> 4. Slicing data by ESCS
>  {'SK': [866, 52.36], 'US': [475, 40.46]}


100%|██████████| 10/10 [00:44<00:00,  4.45s/it]


,count,ratio
1,876,52.96
2,857,51.81
3,863,52.18
4,855,51.69
5,852,51.51
6,838,50.67
7,858,51.87
8,847,51.21
9,859,51.93
10,866,52.36


In [ ]:
display(plot_us)

,count,ratio
1,483,41.14
2,475,40.46
3,473,40.29
4,487,41.48
5,474,40.37
6,484,41.23
7,477,40.63
8,463,39.44
9,485,41.31
10,475,40.46


plt.bar
should be plotted